## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, set_default_openai_client, set_default_openai_api
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from openai import AsyncAzureOpenAI
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [5]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [6]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents developed through various methods. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes LLMs and mathematical frameworks to guide the design of adaptive AI agents. PCF enables real-time parameter reconfiguration through combinatorial spaces, allowing agents to dynamically adjust their behavioral traits. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system that introduces a standardized Agent-to-Agent communication layer based on the Model Context Protocol (MCP). It incorporates an Experience Pack architecture for structured knowledge retention and continual learning. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It focuses on curating high-quality training data and enhancing agent robustness and performance. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **Google Antigravity**: An AI-powered integrated development environment (IDE) that introduces an "agent-first" paradigm, allowing developers to delegate complex coding tasks to autonomous AI agents. It features an agent sidebar and a control center for orchestrating multiple agents. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Google_Antigravity?utm_source=openai))

- **OpenClaw**: A free and open-source autonomous AI assistant software that executes tasks via large language models, using messaging platforms as its main user interface. It gained popularity due to its open-source nature and related projects. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenClaw?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Pte Ltd, designed to independently carry out complex real-world tasks without direct or continuous human guidance. It is considered one of the world's first autonomous agents capable of independent thinking and dynamic planning. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

- **Agent 365**: Microsoft's tool introduced to automate tasks, streamline workflows, and operate across different digital environments. It integrates with Microsoft 365 apps and is compatible with AI agents built through Microsoft's tools and open-source options. ([windowscentral.com](https://www.windowscentral.com/microsoft/microsoft-doubles-down-on-agentic-ai-agent-365-prepares-for-a-future-with-over-1-billion-agents?utm_source=openai))

- **Amazon Bedrock AgentCore**: AWS's platform designed to streamline the development and deployment of advanced AI agents. It offers modular components tailored for scalability, memory management, identity handling, interoperability, debugging, and performance monitoring. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Model Context Protocol (MCP)**: A standardized protocol adopted by OpenAI and supported by other major AI organizations, facilitating the integration and interoperability of AI agents across different platforms. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

These frameworks represent significant advancements in AI agent development, each contributing to the evolution of autonomous and collaborative AI systems. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [7]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    output_type=WebSearchPlan,
)

In [8]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To get comprehensive and updated information about the top AI agent frameworks currently available in 2025.', query='best AI agent frameworks 2025'), WebSearchItem(reason='To find recent news, trends, and advancements in AI agent frameworks specifically released or updated in 2025.', query='latest AI agent framework releases 2025'), WebSearchItem(reason='To see community and expert discussions or rankings about the most popular and widely used AI agent frameworks in 2025.', query='most popular AI agent frameworks 2025')]


In [9]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("ed.donner@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"

In [10]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000022BB03C3CE0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [11]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [12]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [13]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [14]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [15]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


In [16]:
print(report)


short_summary="In 2025, the AI agent ecosystem expanded dramatically, with leading tech companies and open-source communities introducing new frameworks that broaden the capabilities, accessibility, and security of autonomous systems. Notable highlights include Agent Lightning's RL-based structure, enterprise-ready solutions by Salesforce and Microsoft, strong open-source showings like Cognitive Kernel-Pro and OpenClaw, and experimental environments focusing on security and collaborative multi-agent architectures. Comparative studies across these frameworks reveal varied strengths, suggesting that the landscape is increasingly suited for specialized, large-scale, and trustworthy AI deployment." markdown_report='# The Latest AI Agent Frameworks in 2025: A Comprehensive Review\n\n## Table of Contents\n1. Introduction\n2. Evolution of AI Agent Frameworks until 2025\n3. Overview of New and Leading Frameworks\n    - Agent Lightning\n    - GoalfyMax\n    - AgentScope 1.0\n    - Cognitive Ker

In [19]:
import IPython
from IPython.display import Markdown, display

display(Markdown(report.markdown_report))

# The Latest AI Agent Frameworks in 2025: A Comprehensive Review

## Table of Contents
1. Introduction
2. Evolution of AI Agent Frameworks until 2025
3. Overview of New and Leading Frameworks
    - Agent Lightning
    - GoalfyMax
    - AgentScope 1.0
    - Cognitive Kernel-Pro
    - Google Antigravity & ADK
    - Salesforce Agentforce 360
    - Microsoft Agent 365
    - OpenAI Agents SDK
    - Other Notable Frameworks (Manus, Kruti, Agent S2, DoomArena, OpenClaw, etc.)
4. Comparative Analyses and Empirical Studies
5. Key Innovations and Trends
6. Use Cases and Real-World Applications
7. Challenges and Open Problems
8. Future Directions
9. Conclusion
10. References

---

## 1. Introduction

The field of artificial intelligence agents witnessed immense innovation in 2025 as global demand surged for robust, autonomous, and collaborative digital assistants. Frameworks introduced this year pushed the boundaries in reinforcement learning, explainability, scalability, real-world autonomy, multi-agent collaboration, and developer accessibility. This report synthesizes the most State-of-the-Art agent frameworks as of early 2026, providing a roadmap for developers, researchers, and enterprises seeking optimal solutions for diverse AI-driven applications.

## 2. Evolution of AI Agent Frameworks until 2025

Since the early 2020s, AI agent frameworks have evolved from narrow, rule-based assistants into versatile systems capable of independent reasoning, multi-step planning, and adaptive learning. Earlier offerings such as LangChain and Auto-GPT, focusing on LLM chaining and autonomy, laid the groundwork for robust successors. By 2025, frameworks began to deeply integrate reinforcement learning, collaboration protocols, natural language interface design, and security-conscious development, reflecting the maturity of the domain.

## 3. Overview of New and Leading Frameworks

### Agent Lightning
Introduced in August 2025 ([arxiv.org](https://arxiv.org/abs/2508.03680)), Agent Lightning enables reinforcement learning-based training for large language model agents. Its hierarchical RL engine (LightningRL) decouples execution from training, supporting seamless integration with legacy agents (including those built on LangChain or OpenAI's SDK). Features include:
- Formulation of agent behavior as a Markov decision process
- Hierarchical workflow design for complex tasks (e.g., retrieval-augmented generation)
- Multi-agent support and dynamic workflow adaptation

### GoalfyMax
Released July 2025 ([arxiv.org](https://arxiv.org/abs/2507.09497)), this multi-agent framework emphasizes collaboration via:
- Protocol-driven agent-to-agent standardized communication
- Experience Packs for continuous, structured knowledge retention
- Robust adaptability for teams tackling complex, changing tasks

### AgentScope 1.0
Launched in August 2025, AgentScope is developer-centric, focusing on modularity and extensibility. Its key features are:
- Unified, extensible interfaces and modules for building agentic apps
- Systematic asynchronous design for non-blocking interactions
- Flexible plugin and tool integration

### Cognitive Kernel-Pro
Debuted August 2025 ([arxiv.org](https://arxiv.org/abs/2508.00414)), this open-source effort stresses:
- High-quality, multi-domain training data curation (web, code, files, reasoning)
- Test-time agent self-reflection and voting strategies for robustness
- Demonstrated SOTA performance among open-source agents

### Google Antigravity & Agent Development Kit (ADK)
Google's dual offering covers both a cutting-edge AI-powered IDE (Antigravity, November 2025) and a modular agent development kit (ADK, April 2025):
- **Antigravity**: "Agent-first" coding environment, parallel agent orchestration, direct integration with Google Gemini models
- **ADK**: Hierarchical agent composition, minimal-code workflows, native plugs into the Google ecosystem including Vertex AI

### Salesforce Agentforce 360
Announced November 2025, Agentforce 360 is an enterprise-focused platform featuring:
- Agentforce Builder for natural language-driven agent creation
- Seamless Slack integration
- Agentforce Voice for conversational voice-commanded interaction
- Marketplace for packaging and selling custom agents
- 500+ pre-built enterprise features

### Microsoft Agent 365
Debuted December 2025, designed for enterprise workflow automation, with:
- Integration with Copilot Studio and Microsoft Foundry
- Support for both proprietary and open-source agent integration
- AI connectors and secure protocols (Model Context Protocol) for communication

### OpenAI Agents SDK
Lightweight Python framework (March 2025) boasting:
- Multi-agent workflow design, provider-agnosticity
- Compatibility with 100+ LLM backends
- Guardrails, tracing, and multi-level monitoring/debugging

### Other Notable Frameworks
- **Manus**: (Acquired by Meta) True autonomous agent—independent real-world task execution and dynamic planning
- **Kruti**: Indian-origin, multilingual, integrates with daily services (rides, food, etc.), strong in language diversity
- **Agent S2**: Cognitive delegation across generalist/specialist models, proactive planning, strong GUI automation
- **DoomArena**: Security evaluation framework—integrates with other agentic environments for red-teaming and defense
- **OpenClaw**: Open-source AI assistant popularized for its messaging-based UI and extensible open-core architecture

## 4. Comparative Analyses and Empirical Studies

Several studies in 2025 performed head-to-head comparisons and developer experience surveys:
- **Empirical Study of Agent Developer Practices** [arxiv](https://arxiv.org/abs/2512.01939): Compared frameworks on efficiency, abstraction, learning curve, performance optimization, and maintainability. Key findings show AgentScope excels at interface extensibility, Agentforce 360 in enterprise maintainability, and Agent Lightning in RL-driven optimization.
- **Review of Autonomous Systems** [satyadharjoshi.com](https://satyadharjoshi.com/wp-content/uploads/2025/02/Review-of-autonomous-systems-and-collaborative-AI-agent-frameworks-IJSRA-2025-0439.pdf): Provided a use-case-focused matrix comparing LangGraph, CrewAI, OpenAI Swarm, IBM Watsonx.Ai, and others. Open-source frameworks (e.g., Cognitive Kernel-Pro) stood out in research flexibility; commercial platforms (Microsoft, Salesforce) performed best in enterprise scenarios.
- **Performance and Scalability Analysis** [drukarnia.com.ua](https://drukarnia.com.ua/articles/comparing-ai-agent-frameworks-performance-scalability-i-real-world-use-cases-in-2025-e8FVo): Studied CrewAI, SuperAGI, and Haystack Agents across large-scale data processing scenarios and real-time collaboration, highlighting the sophistication of resource orchestration in 2025 frameworks.

## 5. Key Innovations and Trends
- **Hierarchical RL and Reflection**: Agent Lightning, Cognitive Kernel-Pro, and Agent S2 are leaders in integrating advanced RL and meta-reasoning.
- **Multi-Agent Collaboration**: GoalfyMax standardizes multi-agent dialog, and platforms like Antigravity enable collaborative agent orchestration in practical contexts.
- **Security and Governance**: DoomArena, Model Context Protocol (MS), and OpenAI’s guardrails mark the growing importance of testable trust and attack/defense capabilities.
- **Enterprise Integration**: Deep connections with Slack, Microsoft 365, Salesforce, and Google underpin a surge in AI productivity agents.
- **Accessibility and Natural Interface**: Natural language programming, voice-commanded agents, and interface modularity open agent development to wider audiences.

## 6. Use Cases and Real-World Applications
- **Productivity Automation** (Microsoft Agent 365; Salesforce Agentforce 360): Document workflows, meeting management, and routine enterprise automation
- **Complex Research Agents** (Cognitive Kernel-Pro): Web-scale question answering, code evaluation, data mining
- **Real-world Task Execution** (Manus, Kruti): Booking, shopping, logistics via API integrations and action-based planning
- **Developer Tooling** (Antigravity): Delegation of coding tasks, refactoring, test writing
- **Security Testing** (DoomArena): Adversarial agent environments and red-team/blue-team simulation

## 7. Challenges and Open Problems
- **General-Purpose vs. Specialist AI Agents**: Balancing adaptability and depth of domain expertise
- **Trust, Security, and Regulation**: Ensuring agent safety at scale is a moving target—attack surface grows with autonomy
- **Data Quality and Curation**: High-quality, domain-specific datasets remain key to robust agent performance
- **User Experience and Explainability**: Black-box agent decisions still challenge end-user trust and regulatory compliance
- **Scalability in Multi-Agent Systems**: Robust protocols for handling agent-agent communications and task orchestration at scale still require innovation

## 8. Future Directions
- **Federated and Distributed Agent Networks**: Decentralized training, operation, and negotiation
- **Semantic Interoperability Standards**: Common APIs and schema for agent communication
- **Continual and On-Device Learning**: More robust frameworks for privacy-respecting, adaptive agents
- **Explainable Agent Reasoning**: Universal integration of model interpretability and user transparency interfaces

## 9. Conclusion

2025 marked a leap toward comprehensive, interoperable, and accessible agent frameworks supporting a new generation of digital autonomy. While no one-size-fits-all solution has emerged, the increased specialization of frameworks—across research, enterprise, security, and productivity domains—signals the emergence of a mature, application-driven ecosystem. Developers, organizations, and researchers must weigh these nuanced differences to maximize agent potential in their fields.

## 10. References

- [Agent Lightning on arxiv](https://arxiv.org/abs/2508.03680)
- [GoalfyMax on arxiv](https://arxiv.org/abs/2507.09497)
- [AgentScope 1.0 on arxiv](https://arxiv.org/abs/2508.16279)
- [Cognitive Kernel-Pro on arxiv](https://arxiv.org/abs/2508.00414)
- [Google Antigravity on Wikipedia](https://en.wikipedia.org/wiki/Google_Antigravity)
- [Salesforce Agentforce 360 on ITPro](https://www.itpro.com/technology/artificial-intelligence/salesforce-just-launched-a-new-catch-all-platform-to-build-enterprise-ai-agents)
- [Microsoft Agent 365 on WindowsCentral](https://www.windowscentral.com/microsoft/microsoft-doubles-down-on-agentic-ai-agent-365-prepares-for-a-future-with-over-1-billion-agents)
- [OpenAI Agents SDK review on JLC News](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared)
- [Review of Autonomous Systems and Collaborative AI Agent Frameworks](https://satyadharjoshi.com/wp-content/uploads/2025/02/Review-of-autonomous-systems-and-collaborative-AI-agent-frameworks-IJSRA-2025-0439.pdf)
- [Performance and Scalability Analysis of AI Agents](https://drukarnia.com.ua/articles/comparing-ai-agent-frameworks-performance-scalability-i-real-world-use-cases-in-2025-e8FVo)
- [Manus on Wikipedia](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29)
- [Kruti on Wikipedia](https://en.wikipedia.org/wiki/Kruti)
- [OpenClaw on Wikipedia](https://en.wikipedia.org/wiki/OpenClaw)
---



### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>